In [2]:
import torch

from diffusers import StableDiffusionPipeline, DPMSolverMultistepScheduler

pipeline = StableDiffusionPipeline.from_pretrained(
    "gsdf/Counterfeit-V2.5", torch_dtype=torch.float16, safety_checker=None
).to("cuda")
pipeline.scheduler = DPMSolverMultistepScheduler.from_config(
    pipeline.scheduler.config, use_karras_sigmas=True
)

text_encoder\model.safetensors not found


In [4]:
# ??pipeline.load_textual_inversion

In [5]:
pipeline.load_textual_inversion(r"D:\RICH\Project\embd", weight_name="EasyNegative.pt")

In [ ]:
??pipeline.load_lora_weights

In [12]:
# a = torch.load(r"D:\RICH\Project\embd\light_and_shadow.safetensors")
pipeline.load_lora_weights(r"D:\RICH\Project\embd", weight_name="light_and_shadow.safetensors")

KeyError: 'to_k_lora.down.weight'

In [ ]:
prompt = "masterpiece, best quality, 1girl, at dusk"
negative_prompt = ("(low quality, worst quality:1.4), (bad anatomy), (inaccurate limb:1.2), "
                   "bad composition, inaccurate eyes, extra digit, fewer digits, (extra arms:1.2), large breasts")

images = pipeline(prompt=prompt, 
    negative_prompt=negative_prompt, 
    width=512, 
    height=768, 
    num_inference_steps=15, 
    num_images_per_prompt=4,
    generator=torch.manual_seed(0)
).images

In [ ]:
# ! wget https://civitai.com/api/download/models/15603 -O light_and_shadow.safetensors
! pwd

In [ ]:
a = torch.load("EasyNegative.safetensors")
a

In [10]:
a = torch.load(r"D:\RICH\Project\embd\light_and_shadow.safetensors", use_safetensors =True)


TypeError: 'use_safetensors' is an invalid keyword argument for load()

In [15]:
#@title Setup
import os, subprocess

def setup():
    install_cmds = [
        ['pip', 'install', 'gradio'],
        ['pip', 'install', 'open_clip_torch'],
        ['pip', 'install', 'clip-interrogator'],
    ]
    for cmd in install_cmds:
        print(subprocess.run(cmd, stdout=subprocess.PIPE).stdout.decode('utf-8'))

# setup()


caption_model_name = 'blip-large' #@param ["blip-base", "blip-large", "git-large-coco"]
clip_model_name = "ViT-H-14/laion2b_s32b_b79k" #@param ["ViT-L-14/openai", "ViT-H-14/laion2b_s32b_b79k"]

import gradio as gr
from clip_interrogator import Config, Interrogator

config = Config()
config.clip_model_name = clip_model_name
config.caption_model_name = caption_model_name
ci = Interrogator(config)

def image_analysis(image):
    image = image.convert('RGB')
    image_features = ci.image_to_features(image)

    top_mediums = ci.mediums.rank(image_features, 5)
    top_artists = ci.artists.rank(image_features, 5)
    top_movements = ci.movements.rank(image_features, 5)
    top_trendings = ci.trendings.rank(image_features, 5)
    top_flavors = ci.flavors.rank(image_features, 5)

    medium_ranks = {medium: sim for medium, sim in zip(top_mediums, ci.similarities(image_features, top_mediums))}
    artist_ranks = {artist: sim for artist, sim in zip(top_artists, ci.similarities(image_features, top_artists))}
    movement_ranks = {movement: sim for movement, sim in zip(top_movements, ci.similarities(image_features, top_movements))}
    trending_ranks = {trending: sim for trending, sim in zip(top_trendings, ci.similarities(image_features, top_trendings))}
    flavor_ranks = {flavor: sim for flavor, sim in zip(top_flavors, ci.similarities(image_features, top_flavors))}
    
    return medium_ranks, artist_ranks, movement_ranks, trending_ranks, flavor_ranks

def image_to_prompt(image, mode):
    ci.config.chunk_size = 2048 if ci.config.clip_model_name == "ViT-L-14/openai" else 1024
    ci.config.flavor_intermediate_count = 2048 if ci.config.clip_model_name == "ViT-L-14/openai" else 1024
    image = image.convert('RGB')
    if mode == 'best':
        return ci.interrogate(image)
    elif mode == 'classic':
        return ci.interrogate_classic(image)
    elif mode == 'fast':
        return ci.interrogate_fast(image)
    elif mode == 'negative':
        return ci.interrogate_negative(image)
        

Loading caption model blip-large...
Loading CLIP model ViT-H-14/laion2b_s32b_b79k...



ViT-H-14_laion2b_s32b_b79k_artists.safetensors:   0%|                                      | 0.00/21.6M [00:00<?, ?B/s]
ViT-H-14_laion2b_s32b_b79k_artists.safetensors:  19%|█████▋                       | 4.19M/21.6M [00:00<00:03, 5.23MB/s]
ViT-H-14_laion2b_s32b_b79k_artists.safetensors:  39%|███████████▎                 | 8.39M/21.6M [00:01<00:01, 8.52MB/s]
ViT-H-14_laion2b_s32b_b79k_artists.safetensors:  58%|████████████████▉            | 12.6M/21.6M [00:01<00:00, 9.02MB/s]
ViT-H-14_laion2b_s32b_b79k_artists.safetensors:  78%|██████████████████████▌      | 16.8M/21.6M [00:01<00:00, 11.4MB/s]
ViT-H-14_laion2b_s32b_b79k_artists.safetensors: 100%|█████████████████████████████| 21.6M/21.6M [00:02<00:00, 9.53MB/s]

ViT-H-14_laion2b_s32b_b79k_flavors.safetensors:   0%|                                       | 0.00/207M [00:00<?, ?B/s]
ViT-H-14_laion2b_s32b_b79k_flavors.safetensors:   2%|▌                             | 4.19M/207M [00:00<00:30, 6.73MB/s]
ViT-H-14_laion2b_s32b_b79k_flavors.saf

Loaded CLIP model and data in 469.77 seconds.


In [16]:
from diffusers import StableDiffusionInpaintPipeline
pipe = StableDiffusionInpaintPipeline.from_pretrained("runwayml/stable-diffusion-inpainting")

text_encoder\model.safetensors not found


Fetching 16 files:   0%|          | 0/16 [00:00<?, ?it/s]

OSError: Consistency check failed: file should be of size 1216061799 but has size 278201428 (pytorch_model.bin).
We are sorry for the inconvenience. Please retry download and pass `force_download=True, resume_download=False` as argument.
If the issue persists, please let us know by opening an issue on https://github.com/huggingface/huggingface_hub.